In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
np.zeros((4,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [19]:
# Variables, parameters, initializations
num_simulations = 1000
N = 1000 # true population size
num_gens = 300
num_pops = 2 # pop A and pop B
mu = [10**-1, 10**-2, 10**-3, 10**-4] # innovation rate
role_models_otm = 0.1
role_models_freqDep=1
theta_otm = 1
theta_freqDep = [0.5,1.0,1.5]

# Store 2-D lists for the Ne, SDI, and number of unique variants for each population PER generation
ne_mtx = np.zeros((num_gens, num_pops))
sdi_mtx = np.zeros((num_gens, num_pops))
unique_vars = np.zeros((num_gens, num_pops))

theta = 1
role_model_prop = 0.5

In [51]:
otm = ChooseModel(role_models_otm, mu, theta_otm, 1, num_gens, N, "OTM")

1


/var/folders/8q/2_j87p0x7y34xjd9jbxfjzrh0000gn/T/ipykernel_6338/896219622.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  ne[0,0] = (pop_size-1) /(np.var(numOffspring[0]))
/var/folders/8q/2_j87p0x7y34xjd9jbxfjzrh0000gn/T/ipykernel_6338/896219622.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  ne[0,1] = (pop_size-1) /(np.var(numOffspring[1]))


TypeError: 'int' object is not subscriptable

In [45]:
def ChooseModel(role_models, mu, theta, num_sims,num_gens,N,model):
    
    #running simulations based on model type
    if model == "OTM":
        
        #list to keep track of results for each innovation rate
        per_mu = []
        for i in mu:
            
            #for OTM, theta = 1, loop on each 0.1 proportion of role models from 0.1->1
            while (role_models*N) <= N:
                
                #list to keep track of results at each role model ratio
                per_rm = []
                
                #initialize ne,sdi,vars arrays per population, per generation
                final_ne = np.zeros([num_gens,num_pops])
                final_sdi = np.zeros([num_gens,num_pops])
                final_vars = np.zeros([num_gens, num_pops])
                
                #for each simulation, get the resulting ne,sdi,vars and add it to final arrays
                for n in range(num_sims):
                    sim_ne, sim_sdi, sim_vars = RunSimulation(num_gens,N,role_models,i,theta)
                    final_ne += sim_ne
                    final_sdi += sim_sdi
                    final_vars += sim_vars
                    
                #average out each array by the number of simulations to get mean values per gen
                final_ne = final_ne/num_sims
                final_sdi = final_sdi/num_sims
                final_vars = final_vars/num_sims
                
                #append a list of all three arrays in the role model list
                #to get the values at this current role model rate
                per_rm.append([final_ne,final_sdi,final_vars])
            
            #finally, append the "per role model rate" list into the list for each mu value
            per_mu.append(per_rm)
        
        #return this list for figure creation
        #contains the final,averaged arrays for ne,sdi,vars across all gens
        #across role model proportions
        return per_mu
    
    
    else:
        #for frequency dependent model,
        #role_models = 1 because the whole population can transmit cultural information
        #looping the same way as the previous except iterating per theta value instead
        #of role models
        per_mu = []
        for i in mu:
            #keep track of results for each value of theta
            per_theta = []
            for t in theta:
                for n in range(num_sims):
                    sim_ne,sim_sdi,sim_vars = RunSimulation(num_gens,N,role_models,i,theta)
                    final_ne += sim_ne
                    final_sdi += sim_sdi
                    final_vars += sim_vars
                    
                #same pattern of averaging across generations per sim
                final_ne = final_ne/num_sims
                final_sdi = final_sdi/num_sims
                final_vars = final_vars/num_sims
                
                #only need final_ne for figures for frequency dependent
                per_theta.append(final_ne)
            
            #append each theta result into per_mu array
            per_mu.append(per_theta)
        
        #return this list for figure 3 creation
        #contains the final, averaged array for Ne across all gens
        #for each combination of theta and mu
        return per_mu
                

In [46]:
def RunSimulation(num_gens, pop_size, role_models, mu, theta):
    #set up each population as an array of N members
    pop = np.zeros([2,pop_size])
    
    pop, max_vars, sdi_a, sdi_b, tmp_pop = RunBurnIn(pop, pop_size, mu)
    
    #set up 2-D arrays for Ne, SDI, and number of unique variants for each pop
    #PER generation
    ne = np.zeros([num_gens, 2])
    sdi = np.zeros([num_gens,2])
    unique_vars = np.zeros([num_gens,2])
    
    #Calculate initial values for ne,sdi,vars after burn-in
    #first, ne...calculate the number of times each variant shows up
    #in the population, and set that as the number of offspring for 
    #that variant. Ne = (popsize -1) / variance of offspring produced
    #per variant
    numOffspring = []
    for pop in range(2):
        popOffspring= np.zeros(pop_size)
        for i in range(pop_size):
            np.append(popOffspring, np.count_nonzero(tmp_pop[0]==i))
        numOffspring.append(popOffspring)
    
    ne[0,0] = (pop_size-1) /(np.var(numOffspring[0]))
    ne[0,1] = (pop_size-1) /(np.var(numOffspring[1]))
    
    print(pop)
    #next, sdi:
    sdi[0,0] = SDI(pop[0])
    sdi[0,1] = SDI(pop[1])
    
    #lastly, number of unique variants
    unique_vars[0,0] = np.unique(pop[0])
    unique_vars[0,1] = np.unique(pop[1])
    
    for gen in range(1, num_gens):
        cur_ne, cur_sdi, cur_vars, pop = RunGeneration(pop, pop_size, theta, role_models)
        ne[i,0] = cur_ne[0]
        ne[i,1] = cur_ne[1]
        
        sdi[i,0] = cur_sdi[0]
        sdi[i,1] = cur_sdi[1]
        
        unique_vars[i,0] = cur_vars[0]
        unique_vars[i,1] = cur_vars[1]
        
        max_vars[0] = pop[0].max()
        max_vars[1] = pop[1].max()
    return ne,sdi,unique_vars
    

In [49]:
def RunBurnIn(pop, pop_size, mu):
    
    pop[0] = np.random.choice(pop_size, pop_size, replace=False)
    pop[1] = np.array([1]*pop_size)
    
    max_vars = []
    max_vars.append(pop_size)
    max_vars.append(1)
    
    sdi_a = SDI(pop[0])
    sdi_b = SDI(pop[1])
    tmp_pop = np.zeros([2,pop_size])
    
    while sdi_a > sdi_b:
        tmp_pop = np.zeros([2,pop_size])
        
        for i in range(2):
            #perform cultural transmission
            tmp_pop[i] = np.random.choice(pop_size, pop_size, replace=True)
            pop[i] = pop[i, tmp_pop[i]]
            
            #undergo innovation
            new_innovators = np.random.binomial(1, mu, pop_size)
            pop[i,np.where(new_innovators==1)] = np.arange(max_vars[i]+1,max_vars[i]+len(np.where(new_innovators ==1))+1)
            max_vars[i] = pop[i].max()
            
        sdi_a = SDI(pop[0])
        sdi_b = SDI(pop[1])
    
    return pop, max_vars, sdi_a, sdi_b, tmp_pop

In [50]:
def RunGeneration(pop, pop_size,theta, role_models):
    ne = np.zeros(2)
    sdi = np.zeros(2)
    unique_vars = np.zeros(2)
    for i in range(2):
        #set of potential people to transmit new vars (only from role models)
        #If FreqDep, then each person has the potential of transmitting a variant
        new_vars = np.random.choice(pop_size,role_models*pop_size, replace=False)
        
        #getting the actual variants they have from the population
        variants = np.unique(pop[i,new_vars])
        
        #getting variant frequencies
        var_freq = np.zeros(len(variants))
        for j in range(len(variants)):
            var_freq[j] = len(np.where(pop[i,new_vars]== variants[j]))
        
        #getting the probability of individuals choosing each variant
        probs = np.power(var_freq,theta) / sum(np.power(var_freq,theta))
        probs_individual = probs/var_freq
        
        #assigning weights
        weights = np.zeros(len(new_vars))
        for v in range(len(new_vars)):
            weights[v] = probs_individual[np.where(variants == pop[i,new_vars[v]])]
        
        #getting new set of culturally transmitted individuals
        tmp = np.random.choice(new_vars, pop_size, replace=True, p=weights)
        
        pop[i] = pop[i,tmp]
        
        #computing ne, sdi, unique vars
        offspring = np.zeros(pop_size)
        for p in range(pop_size):
            offspring[p] = (len(np.where(tmp == p)))
        ne[i] = (pop_size-1) / np.var(offspring)
        
        sdi[i] = SDI(pop[i])
        
        unique_vars[i] = np.unique(pop[i])
        
        #add innovators
        new_innovators = np.random.binomial(1, mu, pop_size)
        pop[i,np.where(new_innovators==1)] = np.arange(max_vars[i]+1,max_vars[i]+len(np.where(new_innovators ==1))+1)
        
    return ne,sdi,unique_vars,pop

In [14]:
#####Finished up to here...

##### I re-arranged it so we weren't calling functions for calculations
##### like "calculating cultural transmission" or "performing innovation"
##### but feel free to change this if the code is too clunky without it

##### functions remaining: SDI()

##### Other things remaining: code to actually call 'ChooseModel',
##### figure creation

In [15]:
def SDI(subPop):
    count = 0
    for i in np.unique(subPop):
        count += len(np.where(subPop == i))
    return (1 - (count/(len(subPop) * (len(subPop)-1))))